![Astrofisica Computacional](../logo.PNG)

---
## 02. Ecuaciones Diferenciales Parciales II. Métodos de Diferencias Finitas. La Ecuación Lineal de Advección. 

Eduard Larrañaga (ealarranaga@unal.edu.co)

---


### Resumen

Se presentan algunos métodos para resolver la ecuación de advección lineal de advección.


---

## La Ecuación Lineal de Advección 

Considere la ecuación de continuidad,

\begin{equation}
\frac{\partial \rho}{\partial t} + \vec{\nabla} \cdot (\rho \vec{v}) = 0
\end{equation}

en el caso particular de un sistema unidimensional y para un fluido que se mueve con velocidad constante,

\begin{equation}
\frac{\partial \rho}{\partial t} + \vec{v}_x \frac{\partial \rho}{\partial x}  = 0.
\end{equation}

Esta es un ejemplo de los **sistemas hiperbólicos de primer orden** en una dimensión espacial y se denomina la **ecuación lineal de advección**. Utilizando la función $\psi=\psi(t,x)$ dentro de esta ecuación, se tiene

\begin{equation}
\partial_t \psi + v \partial_x \psi = 0\,\,,
\end{equation}

donde $v$ es una constante. Una solución exacta de esta ecuación está dada por cualquier función que dependa de los argumentos

\begin{equation}
\psi(t,x) = \psi(x\pm vt)\,\,.
\end{equation}

De esta forma, la ecuación de advección trasladará el perfil inicial dado a lo largo del eje-x con una velocidad constante $v$.

---

## Métodos de Solución para la Ecuación de Advección

### 1. Discretización FTCS 

Este método de diferencias finitas esta basado en discretizar la derivada temporal hacia adelante y la derivada espacial centrada. Su nombre proviene de las siglas en inglés (**F**orward in **T**ime, **C**entered in **S**pace: FTCS).  Esta discretización lleva de inmediato a la relación 

\begin{equation}
  \psi^{(n+1)}_j = \psi^{(n)}_j - \frac{v \Delta t}{2\Delta x} \left(\psi^{(n)}_{j+1} - \psi^{(n)}_{j-1}\right)\,\,.
\end{equation}

En esta ecuación, el índice $n$ hace referencia a la discretización temporal mientras que el índice $j$ se refiere a la discretización espacial. 

**Condiciones de frontera**

Es importante notar que en este método, la definición de la malla necesita de un criterio para tratar las condiciones de frontera. Algunos de los posibles criterios son:

- *Outflow* : la información de los últimos puntos internos se copia en los puntos de frontera. En este caso el perfil sale del dominio de integración sin devolverse.

- *Condición Periodica* : La información de los puntos de la frontera en uno de los extremos se copia en el otro extremo. En esta caso, el perfil sale por uno de los extremos del dominio de integración para re-aparecer por el otro extremo.


**Estabilidad (Criterio de Von Neumann)**

La estabilidad del método se analiza introduciendo la función $\psi(x_j,t_0) = \psi_j^{(0)} = e^{ikx_j} = e^{ikj\Delta x} $ en la ecuación discretizada. Esto da como resultado

\begin{equation}
\begin{aligned}
\psi^{(1)}_j &= e^{ik j\Delta x } - \frac{v \Delta t}{2\Delta x} \left(e^{ik (j+1)\Delta x } - e^{ik (j-1) \Delta x }\right)\,\,,\\
\psi^{(1)}_j &= \left[ 1 - \frac{v\Delta t}{2 \Delta x} \left( e^{ik\Delta x} - e^{-ik\Delta x} \right) \right] e^{ik j \Delta x }\,\,,\\
\psi^{(1)}_j &= \underbrace{\left[ 1- \frac{v\Delta t}{\Delta x} i \sin(k\Delta x) \right]}_{{=\xi}} e^{ik j \Delta x }\,\,,
\end{aligned}
\end{equation}

Después de $n$ iteraciones, 

\begin{equation}
\begin{aligned}
\psi^{(n)}_j &= \left[ 1- \frac{v\Delta t}{\Delta x} i \sin(k\Delta x) \right]^n e^{ik j \Delta x } = \xi^n e^{ik j \Delta x }\,\,.
\end{aligned}
\end{equation}

El coeficiente $\xi$ satisface 

\begin{equation}
|\xi| = \sqrt{\xi \xi^*} = \sqrt{1 + \left(\frac{v\Delta t}{\Delta x} \sin(k\Delta x)\right)^2} > 1\,\,,
\end{equation}

lo que implica que el método FTCS es **incondicionalmente inestable** para la ecuación de advección!

### Ejemplo

La advección es un proceso importante en muchas areas de la astrofísica. Por ejemplo, algunos modelos de transporte en la heliosfera consideran distribuciones de partículas aceleradas por choques (Litvinenko-2014) que son descritas, en primera aproximación, por una ecuación de advección-difusión unidimensional para la densidad de partículas. En la aproximación de difusión débil, la ecuación que se debe resolver es

\begin{equation}
\frac{\partial \psi}{\partial t} + v\frac{\partial \psi}{\partial x} = 0\,\,,
\end{equation}

dodne $v$ representa la velocidad de advección constante que puede ser interpretada como la rapidez del viento solar.

Como un ejemplo para resolver esta ecuacion diferencial parcial, se advectará un perfil sinusoidal,

\begin{equation}
 \Psi_0 = \psi(x,t=0) = \sin \left(\frac{4\pi x}{L} \right),
\end{equation}
 
donde $L = x_f -x_0$ es el tamaño del intervalo epacial de integración. Se considerará  una velocidad positiva $v = 0.1$ en el dominio espacial $x \in [0,100]$.

En este caso se utilizará el criterio de **condición periódica** para las fronteras.

---
### Evolución Temporal de la Solución




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Definition of the initial Sin profile
def sinProfile(x):
    return np.sin(4*np.pi*x/(x_f-x_i))


# Definition of the grids
x_i = 0.
x_f = 100.
Nx = 500
xgrid = np.linspace(x_i, x_f, Nx)


t_i = 0.
t_f = 400.
Nt = 10000
tgrid = np.linspace(t_i, t_f, Nt)


# Velocity of the advection
v = 0.1


# FTCS Method
def FTCS(psi0, tgrid, xgrid, boundary='outflow'):
    print('FTCS Method')
    dt = tgrid[1] - tgrid[0]
    dx = xgrid[1] - xgrid[0]
    print('dt = ', dt)
    print('dx = ', dx)
    
    psi = np.zeros([len(tgrid), len(xgrid)])
    psi[0] = psi0(xgrid) # initial condition
    
    for n in range(len(tgrid)-1):
        for j in range(1,len(xgrid)-1):
            psi[n+1,j] = psi[n,j] - ((v*dt)/(2*dx))*(psi[n,j+1] - psi[n,j-1])
        if boundary=='outflow': # Outflow boundary conditions
            psi[n+1,0] = psi[n+1,1]
            psi[n+1,-1] = psi[n+1,-2]
        else: # Periodic boudnary conditions
            if v>0:
                psi[n+1,0] = psi[n+1,-2]
                psi[n+1,-1] = psi[n+1,-2]
            else:
                psi[n+1,0] = psi[n+1,1]
                psi[n+1,-1] = psi[n+1,0]
    return psi


psi = FTCS(sinProfile, tgrid, xgrid, boundary='periodic')



plt.figure(figsize=(7,5))
plt.title('FTCS Method')
plt.plot(xgrid, psi[0,:], label=f'$t =$ {tgrid[0]:.0f}')
plt.plot(xgrid, psi[-1,:], label=f'$t =$ {tgrid[-1]:.0f}')
plt.xlabel(r'$x$')
plt.ylabel(r'$\psi(t,x)$')
plt.legend()
plt.grid()
plt.show()



---
### Animación de la advección

In [ ]:
from matplotlib import animation
from IPython.display import HTML

%matplotlib inline

# Set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
plt.title('FTCS Method')
ax.set_xlim(( 0, 100))
ax.set_ylim((-1.1, 1.1))
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$\psi(t,x)$')
plt.grid()

line, = ax.plot([], [], lw=2)

In [ ]:
# Define the initialization function, which plots the background of each frame
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    x = xgrid
    y = psi[int(Nt/200)*i,:] # We take only some of the frames (given by 50*i)
    line.set_data(x, y)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20,  # only 200 frames with a duration of 20 ms each
                               blit=True)

In [ ]:
HTML(anim.to_jshtml())


Como se mostró arriba, em método FTCS es **incondicionalmente inestable** para la ecuación de advección. Por ello se produce el mal comportamiento en la parte final de la anterior animación.  

Para ilustrar nuevamente esta inestabilidad, consideraremos ahora la advección de un perfíl sinuosidal con un paso temporal $dt=0.02$ con condiciones de frontera periodicas. Nótese que en el intervalo temporal $[0, 350]$ el comportamiento no muestra inestabilidades, 

In [ ]:
# Definition of the grids
x_i = 0.
x_f = 100.
Nx = 500
xgrid = np.linspace(x_i, x_f, Nx)
dx = xgrid[1] - xgrid[0]


t_i = 0.
t_f = 350.
dt = 0.05
tgrid = np.arange(t_i, t_f, dt)
Nt = len(tgrid)


# Velocity of the advection
v = 0.1


psi = FTCS(sinProfile, tgrid, xgrid, boundary='periodic')


# Set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
plt.title('FTCS Method')

ax.set_xlim(( 0, 100))
ax.set_ylim((-1.1, 1.1))
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$\psi(t,x)$')
plt.grid()

line, = ax.plot([], [], lw=2)

# Define the initialization function, which plots the background of each frame
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    x = xgrid
    step = int(Nt/200)
    y = psi[step*i] # We take only some of the frames (given by 50*i)
    line.set_data(x, y)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20,  # only 200 frames with a duration of 20 ms each
                               blit=True)

HTML(anim.to_jshtml())

Sin embargo, al extender el intervalo temporal a $[0,700]$, la inestabilidad aparece,

In [ ]:
# Definition of the grids
x_i = 0.
x_f = 100.
Nx = 500
xgrid = np.linspace(x_i, x_f, Nx)
dx = xgrid[1] - xgrid[0]


t_i = 0.
t_f = 1500.
dt = 0.05
tgrid = np.arange(t_i, t_f, dt)
Nt = len(tgrid)


# Velocity of the advection
v = 0.1


psi = FTCS(sinProfile, tgrid, xgrid, boundary='periodic')


# Set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
plt.title('FTCS Method')

ax.set_xlim(( 0, 100))
ax.set_ylim((-1.1, 1.1))
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$\psi(t,x)$')
plt.grid()

line, = ax.plot([], [], lw=2)

# Define the initialization function, which plots the background of each frame
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    x = xgrid
    step = int(Nt/200)
    y = psi[step*i] # We take only some of the frames (given by 50*i)
    line.set_data(x, y)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20,  # only 200 frames with a duration of 20 ms each
                               blit=True)

HTML(anim.to_jshtml())

In [ ]:
HTML(anim.to_html5_video())

Otra forma de visualizar el resultado de la advección es mediante un diagrama espacio-temporal

In [ ]:
tplot_i = 0
tplot_f = 500
# Plot
plt.figure(figsize=(25,5))
plt.title('FTCS Method')
plt.imshow(psi[int(tplot_i/dt):int(tplot_f/dt)].T, cmap='inferno', extent=[tplot_i, tplot_f, x_f, x_i])
plt.xlabel(r'$t$')
plt.ylabel(r'$x$')
plt.colorbar()
plt.show()

In [ ]:
tplot_i = 1000
tplot_f = 1500
# Plot
plt.figure(figsize=(25,5))
plt.title('FTCS Method')
plt.imshow(psi[int(tplot_i/dt):int(tplot_f/dt)].T, cmap='inferno', extent=[tplot_i, tplot_f, x_f, x_i])
plt.xlabel(r'$t$')
plt.ylabel(r'$x$')
plt.colorbar()
plt.show()

---

###  2. Método Upwind 

En este método de solución se considera una discretización  de primer orden en el tiempo y de primer orden en el espacio. Para la derivada espacial se tiene dos posibilidades,

\begin{equation}
\begin{aligned}
\frac{\partial \psi}{\partial x} &\approx \frac{1}{\Delta x}(\psi_j - \psi_{j-1}) \hspace{1cm}\text{diferencias finitas hacia la derecha (upwind)}\\
\frac{\partial \psi}{\partial x} &\approx \frac{1}{\Delta x}(\psi_{j+1} - \psi_{j})
\hspace{1cm}\text{diferencias finitas hacia la izquierda (downwind)}
\end{aligned}
\end{equation}

Las dos posibilidades se denominand aproximaciones laterales porque usan información de un lado o del otro del punto $x_j$. Incorporando estas discretizaciones en la ecuación de advección da como resultado

\begin{equation}
\begin{aligned}
\psi_j^{(n+1)} &= \psi_j^{(n)} - \frac{v\Delta t}{\Delta x} \left(\psi_j^{(n)} - \psi_{j-1}^{(n)}\right) \hspace{1cm}\text{(upwind)}\\
\psi_j^{(n+1)} &= \psi_j^{(n)} - \frac{v\Delta t}{\Delta x} \left(\psi_{j+1}^{(n)} - \psi_{j}^{(n)}\right) \hspace{1cm}\text{(downwind)}\,\,.
\end{aligned}
\end{equation}


**Estabilidad**

Un análisis de estabilidad muestra que el método upwind es estable si

\begin{equation}
0 \le \frac{v \Delta t}{\Delta x} \le 1\,\,,
\end{equation}

mientras que el método downwind es estable si

\begin{equation}
-1 \le \frac{v \Delta t}{\Delta x} \le 0\,\,,
\end{equation}

lo cual confirma que para $v > 0$ se debe utilizar el upwind mientras que para $v < 0$ se debe utilizar el downwind.

La condición

\begin{equation}
\alpha = \left|\frac{v\Delta t}{\Delta x}\right| \le 1\,\,
\end{equation}

es una realización matemática de el principio de causalidad: *la propagación de la información (via advección) en un paso de tiempo $\Delta t$ no debe "saltar" adelante más alla del tamaño $\Delta x$ de un intervalo de la malla espacial*.

El criterio $\alpha \le 1$ usualmente se denomina la **condición de Courant-Friedrics-Lewy (CFL)** y se utiliza para determinar el paso temporal permitido dado el tamaño de la malla espacial $\Delta x$. Así, se suele utilizar el paso temporal

\begin{equation}
\Delta t = c_\mathrm{CFL} \frac{\Delta x}{|v|}\,\,,
\end{equation}

donde $c_\mathrm{CFL} \le 1$ se denomina el factor CFL.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
%matplotlib inline

# Definition of the initial Sin profile
def sinProfile(x):
    return np.sin(4*np.pi*x/(x_f-x_i))


# Upwind Method
def upwind(psi0, tgrid, xgrid, boundary='outflow'):
    print('Upwind/Downwind Method')
    dt = tgrid[1] - tgrid[0]
    dx = xgrid[1] - xgrid[0]
    print('dt = ', dt)
    print('dx = ', dx)
    
    psi = np.zeros([len(tgrid), len(xgrid)])
    psi[0] = psi0(xgrid) # initial condition
    if v>0:
        for n in range(len(tgrid)-1):
            for j in range(1,len(xgrid)-1):
                psi[n+1,j] = psi[n,j] - (v*dt/dx)*(psi[n,j] - psi[n,j-1])
            if boundary=='outflow': # Outflow boundary conditions
                psi[n+1,0] = 0#psi[n+1,1]
                psi[n+1,-1] = psi[n+1,-2]
            else: # Periodic boundary conditions
                psi[n+1,0] = psi[n+1,-2]
                psi[n+1,-1] = psi[n+1,-2]
    else:
        for n in range(len(tgrid)-1):
            for j in range(1,len(xgrid)-1):
                psi[n+1,j] = psi[n,j] - (v*dt/dx)*(psi[n,j+1] - psi[n,j])
            if boundary=='outflow': # Outflow boundary conditions
                psi[n+1,0] = psi[n+1,1]
                psi[n+1,-1] = 0#psi[n+1,-2]
            else: # Periodic boundary conditions
                psi[n+1,0] = psi[n+1,1]
                psi[n+1,-1] = psi[n+1,0]
                
    return psi


# Velocity of the advection
v = 0.1

# CFL Coefficient
C = 0.2

# Definition of the grids
x_i = 0.
x_f = 100.
xgrid = np.linspace(x_i, x_f, 500)
dx = xgrid[1] - xgrid[0]

t_i = 0.
t_f = 1500.
dt = C*dx/abs(v)
tgrid = np.arange(t_i, t_f, dt)

print('C_{CFL} =', C)


psi = upwind(sinProfile, tgrid, xgrid, boundary='periodic')

# Set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
plt.title('Upwind/Downwind Method')
ax.set_xlim((0, 100))
ax.set_ylim((-1.1, 1.1))
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$\psi(t,x)$')
plt.grid()

line, = ax.plot([], [], lw=2)

# Define the initialization function, which plots the background of each frame
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    x = xgrid
    step = int(len(tgrid)/200)
    y = psi[step*i,:] # We take only some of the frames (given by 50*i)
    line.set_data(x, y)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20,  # only 200 frames with a duration of 20 each
                               blit=True)

HTML(anim.to_jshtml())

In [ ]:
tplot_i = 0
tplot_f = 1500
# Plot
plt.figure(figsize=(25,5))
plt.title('Upwind/Downwind Method')
plt.imshow(psi[int(tplot_i/dt):int(tplot_f/dt)].T, cmap='inferno', extent=[tplot_i, tplot_f, x_f, x_i])
plt.xlabel(r'$t$')
plt.ylabel(r'$x$')
plt.colorbar()
plt.show()

---
### 3. Método Lax-Friedrich

El método Lax-Friedrichs, al igual que el FTCS, es de primer orden en el tiempo y segundo orden en el espacio. La ecuación correspondiente es

\begin{equation}
\psi_{j}^{(n+1)} = \frac{1}{2}\left(\psi^{(n)}_{j+1} + \psi^{(n)}_{j-1}\right) - 
\frac{v \Delta t}{2\Delta x} \left(\psi^{(n)}_{j+1} - \psi^{(n)}_{j-1}\right)\,\,,
\end{equation}

de donde se puede comprobar que este método difiere del método FTCS en que se ha logrado la estabilidad para $\alpha \le 1$ al utilizar el promedio del antiguo valor (i.e. en $n$) en los puntos $j+1$ y $j-1$ para calcular el valor actualizado (i.e, en $n+1$) en el punto $j$. 

Es posible demostrar que este tratamiento es equivalente a incluir un termino disipativo en las ecuaciones (que se encarga de amortiguar la inestabilidad del método FTCS) y por ello conelleva a una menor precisión que se manifiesta como una disipación del perfil inicial.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
%matplotlib inline

# Definition of the initial Sin profile
def sinProfile(x):
    return np.sin(4*np.pi*x/(x_f-x_i))


# Lax-Friedrich Method
def LaxF(psi0, tgrid, xgrid, boundary='outflow'):
    print('Lax-Friedrich Method')
    dt = tgrid[1] - tgrid[0]
    dx = xgrid[1] - xgrid[0]
    print('dt = ', dt)
    print('dx = ', dx)
    
    psi = np.zeros([len(tgrid), len(xgrid)])
    psi[0] = psi0(xgrid) # initial condition
    
    for n in range(len(tgrid)-1):
        for j in range(1,len(xgrid)-1):
            psi[n+1,j] = 0.5*(psi[n,j+1] + psi[n,j-1]) - ((v*dt)/(2*dx))*(psi[n,j+1] - psi[n,j-1])
        if boundary=='outflow': # Outflow boundary conditions
            psi[n+1,0] = psi[n+1,1]
            psi[n+1,-1] = psi[n+1,-2]
        else: # Periodic boudnary conditions
            if v>0:
                psi[n+1,0] = psi[n+1,-2]
                psi[n+1,-1] = psi[n+1,-2]
            else:
                psi[n+1,0] = psi[n+1,1]
                psi[n+1,-1] = psi[n+1,0]
    return psi


# Velocity of the advection
v = 0.1

# CFL Coefficient
C = 0.2

# Definition of the grids
x_i = 0.
x_f = 100.
xgrid = np.linspace(x_i, x_f, 500)
dx = xgrid[1] - xgrid[0]

t_i = 0.
t_f = 1500.
dt = C*dx/abs(v)
tgrid = np.arange(t_i, t_f, dt)

print('C_{CFL} =', C)

psi = LaxF(sinProfile, tgrid, xgrid, boundary='periodic')

# Set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
plt.title('Lax-Friedrich Method')
ax.set_xlim((0, 100))
ax.set_ylim((-1.1, 1.1))
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$\psi(t,x)$')
plt.grid()

line, = ax.plot([], [], lw=2)

# Define the initialization function, which plots the background of each frame
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    x = xgrid
    step = int(len(tgrid)/200)
    y = psi[step*i,:] # We take only some of the frames (given by 50*i)
    line.set_data(x, y)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20,  # only 200 frames with a duration of 20 each
                               blit=True)

HTML(anim.to_jshtml())

In [ ]:
tplot_i = 0
tplot_f = 1500
# Plot
plt.figure(figsize=(25,5))
plt.title('Upwind/Downwind Method')
plt.imshow(psi[int(tplot_i/dt):int(tplot_f/dt)].T, cmap='inferno', extent=[tplot_i, tplot_f, x_f, x_i])
plt.xlabel(r'$t$')
plt.ylabel(r'$x$')
plt.colorbar()
plt.show()

--- 
### 4. Método Leap-Frog 

El método **Leap Frog (LF)** (también llamado método del punto medio) es de segundo orden tanto en el tiempo como en el espacio. Éste viene dado por la ecuación

\begin{equation}
\psi_j^{(n+1)} = \psi_j^{(n-1)} - \frac{v \Delta t}{\Delta
x} \left(\psi^{(n)}_{j+1} - \psi^{(n)}_{j-1}\right)\,\,.
\end{equation}

Es posible demostrar que este método es estable para $\alpha < 1$ y se caracteriza porque **no es disipativo**. Esto significa que el perfil inicial se traslada sin cambiar pero un estudio detallado muestra que los modos que componen el perfil no se desplazan con la misma velocidad, lo cual puede llevar a oscilaciones de alta frecuencia  que no se amortiguan (no se incluye una "viscosidad numérica").

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
%matplotlib inline

# Definition of the initial Sin profile
def sinProfile(x):
    return np.sin(4*np.pi*x/(x_f-x_i))

# FTCS 1-step 
def FTCS(psi0, tgrid, xgrid, boundary='outflow'):
    dt = tgrid[1] - tgrid[0]
    dx = xgrid[1] - xgrid[0]
    
    psi = np.zeros([len(tgrid), len(xgrid)])
    psi[0] = psi0(xgrid) # initial condition
    
    for n in range(len(tgrid)-1):
        for j in range(1,len(xgrid)-1):
            psi[n+1,j] = psi[n,j] - ((v*dt)/(2*dx))*(psi[n,j+1] - psi[n,j-1])
        if boundary=='outflow': # Outflow boundary conditions
            psi[n+1,0] = psi[n+1,1]
            psi[n+1,-1] = psi[n+1,-2]
        else: # Periodic boudnary conditions
            if v>0:
                psi[n+1,0] = psi[n+1,-2]
                psi[n+1,-1] = psi[n+1,-2]
            else:
                psi[n+1,0] = psi[n+1,1]
                psi[n+1,-1] = psi[n+1,0]
    return psi


# Leap-Frog Method
def LeapFrog(psi0, tgrid, xgrid, boundary='outflow'):
    print('Leap-Frog Method')
    dt = tgrid[1] - tgrid[0]
    dx = xgrid[1] - xgrid[0]
    print('dt = ', dt)
    print('dx = ', dx)
    
    psi = np.zeros([len(tgrid), len(xgrid)])
    
    # initial condition including one FTCS step
    psi[0:2] = FTCS(psi0, [tgrid[0],tgrid[1]], xgrid, boundary=boundary) 
    
    for n in range(1,len(tgrid)-1):
        for j in range(1,len(xgrid)-1):
            psi[n+1,j] = psi[n-1,j] - (v*dt/dx)*(psi[n,j+1] - psi[n,j-1])
        if boundary=='outflow': # Outflow boundary conditions
            psi[n+1,0] = psi[n+1,1]
            psi[n+1,-1] = psi[n+1,-2]
        else: # Periodic boudnary conditions
            if v>0:
                psi[n+1,0] = psi[n+1,-2]
                psi[n+1,-1] = psi[n+1,-2]
            else:
                psi[n+1,0] = psi[n+1,1]
                psi[n+1,-1] = psi[n+1,0]
    return psi



# Velocity of the advection
v = 0.1

# CFL Coefficient
C = 0.2

# Definition of the grids
x_i = 0.
x_f = 100.
xgrid = np.linspace(x_i, x_f, 500)
dx = xgrid[1] - xgrid[0]

t_i = 0.
t_f = 1500.
dt = C*dx/abs(v)
tgrid = np.arange(t_i, t_f, dt)

print('C_{CFL} =', C)

psi = LeapFrog(sinProfile, tgrid, xgrid, boundary='periodic')

# Set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
plt.title('Leap-Frog Method')
ax.set_xlim((0, 100))
ax.set_ylim((-1.1, 1.1))
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$\psi(t,x)$')
plt.grid()

line, = ax.plot([], [], lw=2)

# Define the initialization function, which plots the background of each frame
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    x = xgrid
    step = int(len(tgrid)/200)
    y = psi[step*i,:] # We take only some of the frames (given by 50*i)
    line.set_data(x, y)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20,  # only 200 frames with a duration of 20 each
                               blit=True)

HTML(anim.to_jshtml())

In [ ]:
tplot_i = 0
tplot_f = 1500
# Plot
plt.figure(figsize=(25,5))
plt.title('Upwind/Downwind Method')
plt.imshow(psi[int(tplot_i/dt):int(tplot_f/dt)].T, cmap='inferno', extent=[tplot_i, tplot_f, x_f, x_i])
plt.xlabel(r'$t$')
plt.ylabel(r'$x$')
plt.colorbar()
plt.show()

---

### 5.Método Lax-Wendroff 

El método **Lax-Wendroff** es una extensión de método Lax-Friedrich haciendolo de segundo orden en el tiempo y en el espacio. La ecuación que lo define es

\begin{equation}
\psi_j^{(n+1)} = \psi_j^n - \frac{v\Delta t}{2\Delta x} \left(\psi_{j+1}^{(n)}
- \psi_{j-1}^{(n)} \right) + \frac{v^2(\Delta t)^2}{2(\Delta x)^2} \left(
\psi_{j-1}^{(n)} - 2 \psi_j^{(n)} + \psi_{j+1}^{(n)}\right)\,\,.
\end{equation}

Este método posee una mejor precisión que el método Lax-Friedrich y es estable para $\alpha \le 1$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
%matplotlib inline

# Definition of the initial Sin profile
def sinProfile(x):
    return np.sin(4*np.pi*x/(x_f-x_i))


# Lax-Wendroff Method
def LaxW(psi0, tgrid, xgrid, boundary='outflow'):
    print('Lax-Wendroff Method')
    dt = tgrid[1] - tgrid[0]
    dx = xgrid[1] - xgrid[0]
    print('dt = ', dt)
    print('dx = ', dx)
    
    psi = np.zeros([len(tgrid), len(xgrid)])
    psi[0] = psi0(xgrid) # initial condition
    
    for n in range(len(tgrid)-1):
        for j in range(1,len(xgrid)-1):
            psi[n+1,j] = psi[n,j] - ((v*dt)/(2*dx))*(psi[n,j+1] - psi[n,j-1])\
                         + 0.5*(v*dt/dx)**2*(psi[n,j+1] - 2*psi[n,j] + psi[n,j-1])
        if boundary=='outflow': # Outflow boundary conditions
            psi[n+1,0] = psi[n+1,1]
            psi[n+1,-1] = psi[n+1,-2]
        else: # Periodic boudnary conditions
            if v>0:
                psi[n+1,0] = psi[n+1,-2]
                psi[n+1,-1] = psi[n+1,-2]
            else:
                psi[n+1,0] = psi[n+1,1]
                psi[n+1,-1] = psi[n+1,0]
    return psi


# Velocity of the advection
v = 0.1

# CFL Coefficient
C = 0.2

# Definition of the grids
x_i = 0.
x_f = 100.
xgrid = np.linspace(x_i, x_f, 500)
dx = xgrid[1] - xgrid[0]

t_i = 0.
t_f = 1500.
dt = C*dx/abs(v)
tgrid = np.arange(t_i, t_f, dt)

print('C_{CFL} =', C)

psi = LaxW(sinProfile, tgrid, xgrid, boundary='periodic')

# Set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
plt.title('Lax-Wendroff Method')
ax.set_xlim((0, 100))
ax.set_ylim((-1.1, 1.1))
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$\psi(t,x)$')
plt.grid()

line, = ax.plot([], [], lw=2)

# Define the initialization function, which plots the background of each frame
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    x = xgrid
    step = int(len(tgrid)/200)
    y = psi[step*i,:] # We take only some of the frames (given by 50*i)
    line.set_data(x, y)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20,  # only 200 frames with a duration of 20 each
                               blit=True)

HTML(anim.to_jshtml())

In [ ]:
tplot_i = 0
tplot_f = 1500
# Plot
plt.figure(figsize=(25,5))
plt.title('Lax Method')
plt.imshow(psi[int(tplot_i/dt):int(tplot_f/dt)].T, cmap='inferno', extent=[tplot_i, tplot_f, x_f, x_i])
plt.xlabel(r'$t$')
plt.ylabel(r'$x$')
plt.colorbar()
plt.show()